# Potential Module

The `potential` module provides force field potential functions for bonds, angles, dihedrals, and pair interactions.

## Potential Types

- **Bond potentials**: Harmonic bonds
- **Angle potentials**: Harmonic angles
- **Pair potentials**: Lennard-Jones, Coulombic


In [ ]:
# The potential module provides force field potential functions
# for bonds, angles, dihedrals, and pair interactions.
#
# Example usage:
# from molpy.potential.bond import Harmonic
# from molpy.potential.angle import Harmonic
# from molpy.potential.pair import LJ126, CoulCut
#
# bond_pot = Harmonic(k=100.0, r0=1.5)
# angle_pot = Harmonic(k=50.0, theta0=109.47)
# lj_pot = LJ126(epsilon=0.1, sigma=3.4)
# coul_pot = CoulCut()

print("Potential module provides force field functions")